<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию клиентов страховой компании. При преобразовании качество моделей машинного обучения не должно ухудшиться.

**Описание данных**

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# загружаем данные
try:
    df = pd.read_csv('../01-data/insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head(3)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


<div class="alert alert-block alert-info">
Если мы планируем использовать эти данные для обучении моделей, можно отметить, что в данных нет пропусков, но необходимо будет решать вопрос с дисбалансом классов, т.к. страховые выплаты получали меньше 12% клиентов. 
    
    
</div>

In [18]:
df['Страховые выплаты'].value_counts(normalize=True)

0    0.8872
1    0.0846
2    0.0230
3    0.0036
4    0.0014
5    0.0002
Name: Страховые выплаты, dtype: float64

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос**: Изменятся ли существенно предсказания модели линейной регрессии, если матрицу признаков, используемую при обучении, умножить на случайную обратимую матрицу?

**Ответ:** Предсказания не изменятся

**Обоснование:** 

Пусть $X = XP$, где $P$ - обратимая матрица, тогда

$$
a' = XPw', где \\
w' = ((XP)^T*XP)^{-1}(XP)^Ty
$$

Докажем, что $a = Xw = XPw' = a'$

Имеем
$$
a' = XP((XP)^T*XP)^{-1}(XP)^Ty = XP(P^TX^TXP)^{-1}P^TX^Ty
$$
Поскольку $P$ - обратима по определению, мы можем раскрыть скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$
$$
a' = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = \\
= XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = \\
= XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty = a
$$
Таким образом, утверждение доказано.

## Алгоритм преобразования

**Алгоритм**

Для деперсонификации данных клиентов страховой компании

1. Генерируем случайную матрицу P размера NxN, зафиксировав случайное состояние.
2. Умножаем матрицу признаков X на P, заносим результат в переменную X1.
3. Проверяем соответствие векторов предсказаний в линейной регрессии при обучении на X и X1.


**Обоснование**

В качестве случайной матрицы создадим квадратную матрицу (понятие обратимости существует только для квадратных матриц) размера $n$, где $n$ - количество столбцов первоначальной матрицы признаков.

## Проверка алгоритма

Подготовим выборки для обучения

In [6]:
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

X.shape, y.shape

((5000, 4), (5000,))

Создадим матрицу из нормально распределенных случайных значений.

In [7]:
# задаем случайное состояние 
np.random.seed = 666

# задаем размер матрицы
p_dim = X.shape[1]

P = np.random.normal(size=(p_dim, p_dim))

Проверяем полученную матрицу на обратимость.

In [12]:
np.linalg.inv(P)

array([[-0.63296307, -0.0157622 ,  0.15091792, -0.03874208],
       [ 2.45843046, -1.6971782 , -0.78007046, -0.25825335],
       [ 1.42172216, -1.53039254, -1.1274211 ,  0.43163553],
       [ 1.22640703, -1.18210837, -1.05830714, -0.12503686]])

<div class="alert alert-block alert-info">
Получили обратную матрицу, можно двигаться дальше.
</div>

Умножаем матрицу признаков на созданную матрицу

In [8]:
X1 = X.dot(P)
X1.shape

(5000, 4)

Создаем модели линейной регрессии.

In [9]:
model_X = LinearRegression().fit(X, y)
model_X1 = LinearRegression().fit(X1, y)

pred_X = model_X.predict(X)
pred_X1 = model_X1.predict(X1)

print(r2_score(y, pred_X))
print(r2_score(y, pred_X1))

np.isclose(r2_score(y, pred_X), r2_score(y, pred_X1))

0.42494550286668
0.4249455028666972


True

Таким образом, мы доказали и проверили на практике, что клиентскую информацию можно деперсонифицировать с помощью умножения матрицы признаков на матрицу ранодомных значений, и что полученные признаки не влияют на процесс машинного обучения.